In [ ]:
# -*- coding: utf-8 -*-
import sys
import os

# Phần này để đảm bảo PyTorch có thể tìm thấy thư viện FFmpeg
# Bạn có thể cần điều chỉnh đường dẫn này cho đúng với máy của mình
if sys.platform == 'win32':
    os.environ["PATH"] += os.pathsep + r"C:\Users\Admin\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg.Shared_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-6.1.1-full_build-shared\bin"
    from torchaudio._extension.utils import _init_dll_path
    _init_dll_path()

import torchcodec
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from collections import defaultdict
from IPython.display import display, clear_output
import time

# ===================================================================
# Phần 1: Tải dữ liệu (Giữ nguyên)
# ===================================================================
print("Đang tải dataset (sử dụng cache nếu có)...")
ds = load_dataset("asthalochan/American_Sign_Language")
train_ds = ds['train']
print("Tải dữ liệu hoàn tất!")
print(f"Số lượng video trong tập huấn luyện: {len(train_ds)}")

# ===================================================================
# Phần 2: Tìm hai video có cùng một nhãn (Giữ nguyên)
# ===================================================================
print("\nBắt đầu tìm 2 video có cùng nhãn để so sánh...")

label_to_indices = defaultdict(list)
for i, item in enumerate(train_ds):
    label_to_indices[item['label']].append(i)

video1_idx, video2_idx = -1, -1
target_label = -1

for label, indices in label_to_indices.items():
    if len(indices) >= 2:
        video1_idx = indices[0]
        video2_idx = indices[1]
        target_label = label
        break

if video1_idx == -1:
    print("Không tìm thấy nhãn nào có ít nhất 2 video để so sánh.")
else:
    print(f"Đã tìm thấy 2 video có cùng nhãn '{target_label}'.")
    print(f" - Video 1 có chỉ số (index): {video1_idx}")
    print(f" - Video 2 có chỉ số (index): {video2_idx}")

    # ===================================================================
    # Phần 3: Lấy dữ liệu và hiển thị từng cặp khung hình (ĐÃ THAY ĐỔI)
    # ===================================================================
    
    video_item1 = train_ds[video1_idx]
    video_item2 = train_ds[video2_idx]

    video_tensor1 = video_item1['video'][:]
    video_tensor2 = video_item2['video'][:]

    video_np1 = video_tensor1.numpy()
    video_np2 = video_tensor2.numpy()
    
    num_frames1 = video_np1.shape[0]
    num_frames2 = video_np2.shape[0]
    print(f"\nSố khung hình của Video 1: {num_frames1}")
    print(f"Số khung hình của Video 2: {num_frames2}")

    max_frames = max(num_frames1, num_frames2)

    print(f"\nĐang hiển thị từng cặp khung hình cho nhãn: {target_label}...")

    # Tạo figure lớn để chứa tất cả các cặp frame
    # hoặc hiển thị từng cặp độc lập để tránh crash nếu quá nhiều frame
    # Với mục đích dễ nhìn, mình sẽ hiển thị từng cặp riêng lẻ.

    for i in range(max_frames):
        fig, axes = plt.subplots(1, 2, figsize=(10, 5)) # Một hàng, hai cột cho mỗi cặp

        # --- Hiển thị khung hình của Video 1 ---
        ax1 = axes[0]
        if i < num_frames1:
            frame1 = video_np1[i]
            frame1_for_display = frame1.transpose(1, 2, 0)
            ax1.imshow(frame1_for_display)
            ax1.set_title(f"Video 1 - Frame {i}")
        else:
            ax1.set_title(f"Video 1 - Hết Frame")
            ax1.text(0.5, 0.5, "Hết video", horizontalalignment='center', verticalalignment='center', transform=ax1.transAxes, fontsize=12, color='gray')
        ax1.axis('off')

        # --- Hiển thị khung hình của Video 2 ---
        ax2 = axes[1]
        if i < num_frames2:
            frame2 = video_np2[i]
            frame2_for_display = frame2.transpose(1, 2, 0)
            ax2.imshow(frame2_for_display)
            ax2.set_title(f"Video 2 - Frame {i}")
        else:
            ax2.set_title(f"Video 2 - Hết Frame")
            ax2.text(0.5, 0.5, "Hết video", horizontalalignment='center', verticalalignment='center', transform=ax2.transAxes, fontsize=12, color='gray')
        ax2.axis('off')
        
        plt.suptitle(f"So sánh nhãn '{target_label}' - Cặp Frame {i}", fontsize=14)
        plt.tight_layout(rect=[0, 0, 1, 0.95]) # Điều chỉnh layout để tiêu đề không bị che
        plt.show()

        # Nếu bạn chạy trong môi trường như Jupyter/Colab, bạn có thể muốn tạm dừng
        # một chút giữa các cặp khung hình để dễ xem hơn.
        # time.sleep(0.5) 
        # clear_output(wait=True) # Để xóa hình cũ và vẽ hình mới, chỉ dùng trong Jupyter/Colab